# Flexibility Algo

## Algorithm to Automate Flexibility Scoring

### Import Packages

In [2]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import openpyxl
import xlsxwriter

from nltk.stem import WordNetLemmatizer
import string

from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.feature_extraction.text import TfidfVectorizer

from yellowbrick.cluster import KElbowVisualizer

from nltk.cluster.kmeans import KMeansClusterer

from scipy import stats
from scipy.stats import pearsonr

### Put Data from Excel Sheet into Dataframes

In [2]:
# individual df's for each sheet

# when on pc
data_test_cup = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_cup_semdis.csv")
data_test_key = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_key_semdis.csv")
data_test_rope = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_rope_semdis.csv")
data_test_brick = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_brick_semdis.csv")
data_test_chair = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_chair_semdis.csv")
data_test_pencil = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_pencil_semdis.csv")
data_test_shoe = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_shoe_semdis.csv")

# when on mac
# data_test_cup = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_cup_semdis.csv.xlsx")
# data_test_key = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_key_semdis.csv.xlsx")
# data_test_rope = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_rope_semdis.csv.xlsx")
# data_test_brick = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_brick_semdis.csv.xlsx")
# data_test_chair = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_chair_semdis.csv.xlsx")
# data_test_pencil = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_pencil_semdis.csv.xlsx")
# data_test_shoe = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_shoe_semdis.csv.xlsx")

In [14]:
# when on pc
data_official_cup = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results_cleaned.xlsx", sheet_name='Cup')
data_official_key = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results_cleaned.xlsx", sheet_name='Key')
data_official_rope = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results_cleaned.xlsx", sheet_name='Rope')
data_official_brick = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results_cleaned.xlsx", sheet_name='Brick')
data_official_chair = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results_cleaned.xlsx", sheet_name='Chair')
data_official_pencil = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results_cleaned.xlsx", sheet_name='Pencil')
data_official_shoe = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results_cleaned.xlsx", sheet_name='Shoe')
data_official_box = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results_cleaned.xlsx", sheet_name='Box')

# # when on mac
# # data_official_cup = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results.xlsx", sheet_name='Cup')
# # data_official_key = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results.xlsx", sheet_name='Key')
# # data_official_rope = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results.xlsx", sheet_name='Rope')
# # data_official_brick = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results.xlsx", sheet_name='Brick')
# # data_official_chair = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results.xlsx", sheet_name='Chair')
# # data_official_pencil = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results.xlsx", sheet_name='Pencil')
# # data_official_shoe = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results.xlsx", sheet_name='Shoe')
# # data_official_box = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/flexibility/official/autdata_flex_results.xlsx", sheet_name='Box')

### Preprocessing

In [3]:
# spacy stop words
stopwords_spacy = STOP_WORDS

In [4]:
stopwords_edited = list(stopwords_spacy)
stopwords_edited.append("thing")
stopwords_edited.append("things")
stopwords_edited.append("use")

In [5]:
# method to clean the responses
def process_text(text, stopwords_list, remove_sw, join_list):
    # tokenize text, lemmanize words, removing punctuation, remove stop words, lowercase all words

    # hardcorded for special situations
    text = re.sub("wedging","wedge", text)
    text = re.sub("exersizing","exercising", text)
    text = re.sub("thrown","throw", text)
    
    text = re.sub("/|-"," ", text)
    text = text.translate(str.maketrans('','',string.punctuation))
    tokens = word_tokenize(text)

    tokens = [w.lower() for w in tokens]
    
    if remove_sw:
        tokens = [word for word in tokens if word not in stopwords_list]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
#         stemmer = PorterStemmer()
#         tokens = [stemmer.stem(t) for t in tokens]

    if join_list:
        tokens = ' '.join(tokens)
 
    return tokens

### General Functions

In [6]:
# method to get a list of participants
def get_id_list(df):
    id_list = df['id'].unique()
    id_list = sorted(id_list)
    return id_list

In [7]:
# method to add a new column
# new column are cleaned responses
def get_cleaned_responses(df, stopwords_list, remove_sw, join_list):
    # id_df = df[df.id == id]
    df_processed = df.copy(deep=True)
    responses = df['response'].tolist()

    # make list of processed responses
    for response in range(len(responses)):
        responses[response] = process_text(responses[response], stopwords_list, remove_sw, join_list)

    # add list as column in df
    df_processed['response_processed'] = responses

    return df_processed

## Flexibility Algo 1
### tf-idf scikit-learn + clustering
### ___ as document

In [158]:
def get_tfidf_vector(num_clusters, responses):
    # initialize CountVectorizer object
    tfidf_vectorizer = TfidfVectorizer(use_idf=True)
    # vectorize the phrases
    tfidf = tfidf_vectorizer.fit_transform(responses)
    
    # elbow method to visualize and find out how many clusters to use
#     visualizer = KElbowVisualizer(KMeans(), k=(1,12), timings=False)
#     visualizer.fit(tfidf.toarray())       
#     visualizer.show()

    # nltk kmeans cosine distance implementation
    number_of_clusters = num_clusters
    kmeans = KMeansClusterer(number_of_clusters, repeats=25, distance=nltk.cluster.util.cosine_distance, avoid_empty_clusters=True)
    assigned_clusters = kmeans.cluster(tfidf.toarray(), assign_clusters=True)

#     scikit-learn euclidean distance implementation
#     kmeans = KMeans(n_clusters = num_clusters).fit(tfidf)
        
    # cluster results scikit-learn
    results = pd.DataFrame()
    results['text'] = responses
#     results['category'] = kmeans.labels_
    results['category'] = assigned_clusters
    
    # create dictionary to organize the clusters with their respective phrases
    results_dict = {k: g["text"].tolist() for k,g in results.groupby("category")}
    
    # df of the clusters and the 
    clusters_df = pd.DataFrame(list(results_dict.items()),columns = ['category','responses']) 
    
    # uncomment to see clusters 
#     print(results_dict)
#     display(clusters_df)
    
    return clusters_df

In [97]:
def get_flexibility_score(flexibility_rating_df, num_clusters, responses):
    clusters_df = get_tfidf_vector(num_clusters, responses)
    
    # create dictionary out of cluster df
    # has clusters and their respective responses
    clusters = dict(zip(clusters_df.category, clusters_df.responses))
    
    flex_df_cleaned = flexibility_rating_df[flexibility_rating_df.response_processed != '']
    participants = get_id_list(flex_df_cleaned)
    participants_responses_list = list(zip(flex_df_cleaned.id, flex_df_cleaned.response_processed))
    
    # get dictionary of each participants responses
    participants_responses_dict = {k: [] for k in participants}
    
    for index in range(len(participants_responses_list)):
        participants_responses_dict[participants_responses_list[index][0]].append(participants_responses_list[index][1])
        
    # get dictionary of responses and their respective dictionary
    responses_cluster_rep = {}
    
    for key in clusters:
        for phrase in clusters[key]:
            responses_cluster_rep[phrase] = key
            
    # get dictionary of participants and clusters their responses existed in
    participants_clusters_apperance = {k: [] for k in participants}
    
    for index in range(len(participants_responses_list)):
        participants_clusters_apperance[participants_responses_list[index][0]].append(responses_cluster_rep[participants_responses_list[index][1]])
        
    # get dic of number of clusters a participants responses are in
    
    participants_clusters_seen = {k: [] for k in participants}
    
    for participant in participants_clusters_seen:
        responses_set = set(participants_clusters_apperance[participant])
        participants_clusters_seen[participant] = len(responses_set)
    
#     print(clusters)
#     print()
#     print(participants_responses_list)
#     print()
#     print(participants_responses_dict)
#     print()
#     print(responses_cluster_rep)
#     print()
#     print(participants_clusters_apperance)
#     print()
#     print(participants_clusters_seen)
    
    # create flexiblity df
    flexibility_df = pd.DataFrame(participants_clusters_seen.items(), columns=['id', 'flexibility'])
    return flexibility_df

In [111]:
def get_flexibility_tfidf_scikit_learn_clustering(df, stopwords_list, num_clusters, remove_sw, join_list):
    flexibility_rating_df = get_cleaned_responses(df, stopwords_list, remove_sw, join_list)
    flexibility_rating_df = flexibility_rating_df[flexibility_rating_df.astype(str)['response_processed'] != '']
    responses = flexibility_rating_df['response_processed'].tolist()
    
    # get id list
    id_list = get_id_list(df)
                
    # add flexibility df
    flexibility_rating_df = get_flexibility_score(flexibility_rating_df, num_clusters, responses)
        
    return flexibility_rating_df

In [159]:
get_flexibility_tfidf_scikit_learn_clustering(data_official_box, stopwords_edited, 15, True, True)

,id,flexibility
0,1087,2
1,1093,3
2,1094,3
3,1102,1
4,1104,2
...,...,...
84,1599,2
85,1603,2
86,1610,3
87,1614,1


## Comparing Algo Results with Human Ratings

In [104]:
prompts_list = ['box', 'brick', 'chair', 'cup', 'key', 'pencil', 'rope', 'shoe']
data_list = [data_official_box, data_official_brick, data_official_chair, data_official_cup, data_official_key, data_official_pencil, data_official_rope, data_official_shoe]

### Collect the Method Results

In [16]:
# create the mean between the two raters columns
for i in range(len(data_list)):
    print(prompts_list[i])
    data_list[i]['flexibility_m'] = data_list[i][['flexibility_1', 'flexibility_2']].mean(axis=1)

box
brick
chair
cup
key
pencil
rope
shoe


In [153]:
# print flexibility results for human rater and method
def print_flexibility_scores():
    flexibility_df_list = []
    for i in range(len(prompts_list)):
        print(prompts_list[i])
        id_list = get_id_list(data_list[i])
        participants_clusters_seen = {k: 0 for k in id_list}
        for participant in participants_clusters_seen:
            id_df = data_list[i][data_list[i].id == participant]
            cluster_apperance = len(id_df['flexibility_m'].unique())
            participants_clusters_seen[participant] = cluster_apperance
        flexibility_df_rater = pd.DataFrame(participants_clusters_seen.items(), columns=['id', 'rating'])
        flexibility_df_method = get_flexibility_tfidf_scikit_learn_clustering(data_list[i], stopwords_edited, 15, True, True)
        df_cd = pd.merge(flexibility_df_rater, flexibility_df_method, how='inner', on = 'id')
        flexibility_df_list.append(df_cd)
        
    return flexibility_df_list

In [155]:
# flex_results_list = print_flexibility_scores()

In [157]:
# print out method results meaned, rerun x times
def print_flexibility_scores_avg(reruns):
    flexibility_df_list = []
    for i in range(len(prompts_list)):
        print(prompts_list[i])
        id_list = get_id_list(data_list[i])
        participants_clusters_seen = {k: 0 for k in id_list}
        df_cd = pd.DataFrame()
        for participant in participants_clusters_seen:
            id_df = data_list[i][data_list[i].id == participant]
            cluster_apperance = len(id_df['flexibility_m'].unique())
            participants_clusters_seen[participant] = cluster_apperance
            df_cd = pd.DataFrame(participants_clusters_seen.items(), columns=['id', 'rating'])
        for y in range(reruns):
            flexibility_df_method = get_flexibility_tfidf_scikit_learn_clustering(data_list[i], stopwords_edited, 15, True, True)
            df_cd = pd.merge(df_cd, flexibility_df_method, how='outer', on = 'id')
        df_cd['method_avg'] = df_cd.iloc[:,2:6].mean(axis=1)
        df_cd = df_cd[['id','rating','method_avg']]
        flexibility_df_list.append(df_cd)
        
    return flexibility_df_list 

In [156]:
# flex_results_avg_list = print_flexibility_scores_avg(5)

In [147]:
# write out the flexibility results
def write_flexibility_results(flex_results_list):
    writer = pd.ExcelWriter('flexibility_methods_results_071421.xlsx', engine='xlsxwriter')
    workbook = writer.book
    
    for i in range(len(prompts_list)):
        flex_results_list[i].to_excel(writer, sheet_name = prompts_list[i], startrow = 0, startcol = 0, index = False)
        
    writer.save()

In [148]:
# write_flexibility_results(flex_results_avg_list)

### Correlations for Each Prompt
#### Between Human Rater and Method

In [151]:
# calculate corrrelation once
def write_flexiblity_corr_oneprompt(results):
    corrs_tuple_list = []
    for i in range(len(prompts_list)):
        corrs_tuple_list.append(stats.pearsonr(results[i]['rating'], results[i]['flexibility']))
        
    result_df = pd.DataFrame(corrs_tuple_list, columns=['corrs', 'pval'])
    
    result_df.index = prompts_list
    
    writer = pd.ExcelWriter('flexibility_methods_corr_results_071421_1time.xlsx', engine='xlsxwriter')
    result_df.to_excel(writer, startrow = 0, startcol=0)
    writer.save()
    
    return result_df

In [2]:
# calculate correlation 5 times
def write_flexibility_corr_allprompt(results, reruns):
    writer = pd.ExcelWriter('flexibility_methods_corr_results_071421_5times.xlsx', engine='xlsxwriter')
    workbook = writer.book

    col = 0
    for y in range(5):
        print(y)
        corrs_tuple_list = []
        results = print_flexibility_scores_avg(reruns)
        for i in range(len(prompts_list)):
            corrs_tuple_list.append(stats.pearsonr(results[i]['rating'], results[i]['method_avg']))

        result_df = pd.DataFrame(corrs_tuple_list, columns=['corrs', 'pval'])

        result_df.index = prompts_list

        result_df.to_excel(writer, startrow = 0, startcol= col)
        col = col + len(result_df.columns) + 2
            
    writer.save()

In [1]:
# sheesh = write_flexiblity_corr_oneprompt(flex_results_list)
# sheesh

In [149]:
# write_flexibility_corr_allprompt(flex_results_list)

0
box


,id,rating,method_avg
0,1087,3,2.25
1,1093,2,2.75
2,1094,2,2.25
3,1102,3,2.50
4,1104,1,1.75
...,...,...,...
84,1599,1,2.75
85,1603,2,3.25
86,1610,2,4.25
87,1614,2,2.25


brick


,id,rating,method_avg
0,1087,3,2.50
1,1093,3,3.75
2,1094,1,4.25
3,1102,2,2.25
4,1104,1,2.25
...,...,...,...
84,1603,3,3.00
85,1610,2,5.00
86,1614,2,2.00
87,1621,3,4.25


chair


...

,id,rating,method_avg
0,1087,4,3.75
1,1093,2,2.00
2,1094,1,2.75
3,1102,2,2.00
4,1104,3,4.00
...,...,...,...
85,1603,2,2.75
86,1610,2,5.50
87,1614,2,2.75
88,1621,2,2.25


rope


,id,rating,method_avg
0,1087,4,3.75
1,1093,3,2.75
2,1094,3,4.00
3,1102,2,3.00
4,1104,2,1.50
...,...,...,...
85,1603,3,2.50
86,1610,1,3.25
87,1614,2,2.25
88,1621,2,3.00


shoe


,id,rating,method_avg
0,1087,3,3.00
1,1093,3,2.50
2,1094,3,2.50
3,1102,3,3.75
4,1104,3,2.25
...,...,...,...
85,1603,2,2.50
86,1610,3,2.75
87,1614,1,1.00
88,1621,2,2.25


Algo Design Brainstorming:

To Do List
- [ ] brainstorm strategy
